# Deploy a web application with Helm

In order to deploy an application with Helm you will need a kubeconfig for the cluster you want to deploy to, a namespace to deploy to, and a containerized version of the application to deploy

In [1]:
from ipywidgets import widgets

app_name_select = widgets.Text(description="Application Name", style=dict(description_width='initial'))
namespace_select = widgets.Text(description="K8s Namespace", style=dict(description_width='initial'))
image_select = widgets.Text(description="Container Image", style=dict(description_width='initial'))
path_select = widgets.Text(value='/',description="URL Path", style=dict(description_width='initial'))
dns_host_select = widgets.Text(description="DNS Hostname", style=dict(description_width='initial'))
internal_port_select =  widgets.BoundedIntText(value='8000',description="Exposed Container Port",style=dict(description_width='initial'), min=1, max=65535, step=1)

display(app_name_select, namespace_select, image_select, path_select, dns_host_select, internal_port_select)

def select_vals(ev):
    global app_name, namespace, path, hostname, fqdn, image_name, internal_port
    app_name = app_name_select.value.lower()
    namespace = namespace_select.value
    image_name = image_select.value
    internal_port = internal_port_select.value
    path = path_select.value
    hostname = dns_host_select.value.lower()
    fqdn = hostname + ".k8s.ucar.edu"
    with output:
        print("Values to use in Helm chart")
        print("Application Name: " + app_name)
        print("K8s Namespace: " + namespace)
        print("Container Image: " + image_name)
        print("Port Exposed on Container: " + str(internal_port))
        print("URL: https://" + fqdn + path)

button = widgets.Button(description="Submit")
output = widgets.Output()
button.on_click(select_vals)
display(button, output)

Text(value='', description='Application Name', style=TextStyle(description_width='initial'))

Text(value='', description='K8s Namespace', style=TextStyle(description_width='initial'))

Text(value='', description='Container Image', style=TextStyle(description_width='initial'))

Text(value='/', description='URL Path', style=TextStyle(description_width='initial'))

Text(value='', description='DNS Hostname', style=TextStyle(description_width='initial'))

BoundedIntText(value=8000, description='Exposed Container Port', max=65535, min=1, style=DescriptionStyle(desc…

Button(description='Submit', style=ButtonStyle())

Output()

In [2]:
def change_values(ev):
    search_text = ["#APP_NAME","#URL_PATH","#FQDN",'#HOST','#IMAGE_NAME','#CONTAINER_PORT']
    replace_text = [app_name,path,fqdn,hostname,image_name,str(internal_port)]
    with open('values_temp.yaml', 'r') as file:
        data = file.read()
        for i in range(len(search_text)):
            data = data.replace(search_text[i], replace_text[i])
    with open('web-app/values.yaml', 'w+') as file:
        file.write(data)
    with chg_output:
        print("Helm Chart Values Updated")
        print(data)

chg_button = widgets.Button(description="Update Helm Values")
chg_output = widgets.Output()
chg_button.on_click(change_values)
display(chg_button, chg_output)

Button(description='Update Helm Values', style=ButtonStyle())

Output()

In [3]:
import os
os.environ["KUBENS"] = namespace
os.environ['KUBECONFIG'] = '/home/ncote/.kube/cislrancher.yml'
!helm install --namespace $KUBENS web-app ./web-app

NAME: web-app
LAST DEPLOYED: Thu Aug  3 07:37:21 2023
NAMESPACE: ncote-test
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [4]:
!helm uninstall --namespace $KUBENS web-app

release "web-app" uninstalled
